In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy . linalg as lng # linear algebra from scipy library
from scipy . spatial import distance # load distance function
from sklearn import preprocessing as preproc # load preprocessing function
from sklearn.impute import IterativeImputer
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler

# seaborn can be used to "prettify" default matplotlib plots by importing and setting as default
import seaborn as sns
sns.set() # Set searborn as default

In [2]:
data = pd.read_csv('case1Data.csv')
y = data['y']
X = data.drop('y', axis=1)
# Get number of observations (n) and number of independent variables (p)
[n, p] = np.shape(X)

print('Missing values in X:', X.isnull().sum().sum())

Missing values in X: 1489


In [3]:
# impute missing values with mean
X_imputed_mean = X.fillna(X.mean())

# check if there are any columns with zero variance
variance = np.var(X_imputed_mean, axis=0)
zero_var_cols = np.where(variance == 0)[0]
print('Zero variance columns:', zero_var_cols)

# drop columns with zero variance
X_imputed_mean = X_imputed_mean.drop(X_imputed_mean.columns[zero_var_cols], axis=1)
print('Columns dropped:', X.columns[zero_var_cols])

# devide the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed_mean, y, test_size=0.2, random_state=42)

# standardize the data
scaler = StandardScaler()
X_train_standardardized = scaler.fit_transform(X_train)
X_test_standardardized = scaler.transform(X_test)

X_standardized = scaler.fit_transform(X_imputed_mean)


Zero variance columns: [96]
Columns dropped: Index(['C_02'], dtype='object')


In [4]:
def ols_solver(X, y):
    betas, res, rnk, s = lng.lstsq(X, y)
    
    return betas, res, rnk, s

def ols_analytical(X, y):
    inner_product = np.linalg.inv(X.T @ X)
    outer_product = X.T @ y
    betas = inner_product @ outer_product
    return betas

# OLS on all data

In [5]:
# Include offset / intercept
off = np.ones(n) 
M = np.c_[off, X_standardized]

# linear solver
beta_solv, res_solv, rnk, s = ols_solver(M, y)
yhat_solv = M @ beta_solv

# analytical solution
beta_ana = ols_analytical(M,y)
yhat_ana = M @ beta_ana

# Manual calculation of residuals for solver
res_solv_man = (y - yhat_solv) ** 2    
mse_solv_man = np.mean(res_solv_man)

# Manual calculation of residuals for analytical
res_ana = (y - yhat_ana) ** 2    
mse_ana = np.mean(res_ana)

print(f'rmse from lstsq: {np.sqrt(res_solv/len(y))}') # lng.lstsq returns sum of squared residuals, so we divide by num of obs to get mean
print(f'rmse from solver: {np.sqrt(mse_solv_man)}')
print(f'rmse from nummerical: {np.sqrt(mse_ana)}')

# analytical version
rss = np.sum(res_ana)
print(f'RSS: {rss}')
tss = np.sum((y - np.mean(y))** 2)
print(f'TSS: {tss}')
r2 = (1 - rss / tss) * 100
print(f'R2: {r2}')

rmse from lstsq: []
rmse from solver: 2.9075161306514707e-12
rmse from nummerical: 2.149390803816044e-10
RSS: 4.6198808275289795e-18
TSS: 520031.4508322642
R2: 100.0


# OLS on train / test set

In [6]:
# Include offset / intercept
off = np.ones(80) #n
M = np.c_[off, X_train_standardardized]

# linear solver
beta_solv, res_solv, rnk, s = ols_solver(M, y_train)
yhat_solv = M @ beta_solv

# analytical solution
beta_ana = ols_analytical(M,y_train)
yhat_ana = M @ beta_ana


In [7]:
# Manual calculation of residuals for solver
res_solv_man = (y_train - yhat_solv) ** 2    
mse_solv_man = np.mean(res_solv_man)

# Manual calculation of residuals for analytical
res_ana = (y_train - yhat_ana) ** 2    
mse_ana = np.mean(res_ana)

print(f'rmse from lstsq: {np.sqrt(res_solv/len(y_train))}') # lng.lstsq returns sum of squared residuals, so we divide by num of obs to get mean
print(f'rmse from solver: {np.sqrt(mse_solv_man)}')
print(f'rmse from nummerical: {np.sqrt(mse_ana)}')

rmse from lstsq: []
rmse from solver: 3.964516476617076e-13
rmse from nummerical: 1042.4911571807145


In [8]:
# analytical version
rss = np.sum(res_ana)
print(f'RSS: {rss}')
tss = np.sum((y_train - np.mean(y_train))** 2)
print(f'TSS: {tss}')
r2 = (1 - rss / tss) * 100
print(f'R2: {r2}')

RSS: 86943025.02399883
TSS: 386897.35805355624
R2: -22371.858030098967


In [9]:
# calculate RSME on test set
off_test = np.ones(20) #n
M_test = np.c_[off_test, X_test_standardardized]
yhat_test = M_test @ beta_solv
res_test = (y_test - yhat_test) ** 2
mse_test = np.mean(res_test)
print(f'rmse on test set solver: {np.sqrt(mse_test)}')

# analytical version
yhat_test_ana = M_test @ beta_ana
res_test_ana = (y_test - yhat_test_ana) ** 2
mse_test_ana = np.mean(res_test_ana)
print(f'rmse on test set analytical: {np.sqrt(mse_test_ana)}')

rmse on test set solver: 34.596118243944055
rmse on test set analytical: 3430.5681400867893


# OLS on principal components

In [10]:
pca_data = pd.read_csv('pca_data.csv')
X_pca = pca_data.drop('y', axis=1)

# take the first 40 principal components
X_40 = X_pca.iloc[:, :40]

[n, p] = np.shape(X_40)

In [11]:
# Include offset / intercept
off = np.ones(n) 
M = np.c_[off, X_40]

# linear solver
beta_solv, res_solv, rnk, s = ols_solver(M, y)
yhat_solv = M @ beta_solv

# analytical solution
beta_ana = ols_analytical(M,y)
yhat_ana = M @ beta_ana

# Manual calculation of residuals for solver
res_solv_man = (y - yhat_solv) ** 2    
mse_solv_man = np.mean(res_solv_man)

# Manual calculation of residuals for analytical
res_ana = (y - yhat_ana) ** 2    
mse_ana = np.mean(res_ana)

print(f'rmse from lstsq: {np.sqrt(res_solv/len(y))}') # lng.lstsq returns sum of squared residuals, so we divide by num of obs to get mean
print(f'rmse from solver: {np.sqrt(mse_solv_man)}')
print(f'rmse from nummerical: {np.sqrt(mse_ana)}')

# analytical version
rss = np.sum(res_ana)
print(f'RSS: {rss}')
tss = np.sum((y - np.mean(y))** 2)
print(f'TSS: {tss}')
r2 = (1 - rss / tss) * 100
print(f'R2: {r2}')

rmse from lstsq: 22.454094689574767
rmse from solver: 22.45409468957477
rmse from nummerical: 22.454094689574767
RSS: 50418.63683283898
TSS: 520031.4508322642
R2: 90.30469469641722
